<div style ="padding: 15px 20px;
    color: #c8102e;
    background: #01216955;             
    display: flex;
    justify-content: space-between;
    align-items: center;">
    <div>
        <img src="flag_mini.png">
    </div>
    <div>
        <h2 style = "padding : 0 10px">3 -Afficher un histogramme qui représente la distribution de l'âge des participants pour toutes les éditions </h2>
    </div>
</div>


In [20]:
import pandas as pd
import os 
import ast
import altair as alt

country = 'GBR'

df_hosts = pd.read_csv(r"{}/archive/olympic_hosts.csv".format(os.getcwd()))
df_results = pd.read_csv(r"{}/archive/olympic_results.csv".format(os.getcwd())) 
df_athletes = pd.read_csv(r"{}/archive/olympic_athletes.csv".format(os.getcwd())) 

## **Récupération de tous les participants anglais aux Jo d'été** 

In [21]:
#fusionner df_results & df_hosts pour filter les JO d'étés, puis filtrer
df = df_results.merge(df_hosts, left_on='slug_game', right_on='game_slug')
df.query("country_3_letter_code == @country and game_season == 'Summer'", inplace=True)

#df_team : récupération des athlètes qui participent en équipe
df_team = df[(df.participant_type == 'GameTeam')]
df_indiv = df[(df.participant_type == 'Athlete')]

print('On récupère {} inscrits'.format(df.shape[0]))
print('{} athlètes aux épreuves individuelles, et {} équipes inscrites aux compétitions par équipe.'.format(len(df_indiv), len(df_team)))


On récupère 5300 inscrits
4402 athlètes aux épreuves individuelles, et 898 équipes inscrites aux compétitions par équipe.


## **Création d'un dataset exploitable**

In [22]:
#suppression des lignes où les participants de l'épreuve par équipe sont inconnus 
df_team = df_team.drop(df_team[df_team['athletes'].isnull()].index)

#extractions des participants par équipe qui étaient dans des listes et ajout aux autres partipants 
df = df_indiv.copy()
liste_participant=[]

for index,row in df_team.iterrows():
    l = ast.literal_eval(row['athletes'])
    for participant in l :
        df_new_line =  pd.DataFrame([[row['slug_game'],row['participant_type'],row['game_year'],row['athletes'],participant[1]]],columns=['slug_game','participant_type', 'game_year','athletes','athlete_url'])
        liste_participant.append(participant[0])
        df = pd.concat([df, df_new_line]) 

#suppression des lignes où l'athlète n'a pas d'identifiant unique / athlete_url ( 3944 entrées)
df.drop(df[df['athlete_url'].isnull()].index, inplace=True)

#éliminations des doublons correspondant aux athlètes présents plusieurs fois la même année ( 3291 entrées )
df.drop_duplicates(subset=['athlete_url','game_year'], keep='first', inplace=True)

#Fusion de notre tableau filtré avec le tableau d'info des athlètes pour obtenir l'année de naissance
df_final = df.merge(df_athletes, left_on='athlete_url', right_on='athlete_url')

#suppression des lignes où la date de naissance est inconnue
df_final.drop(df_final[df_final['athlete_year_birth'].isnull()].index, inplace=True)

print("Après un nettoyage de donnée, nous obtenons une liste de {} présences d'athlètes toutes olympiades confondues, dont la date de naissance est connue; ce qui nous permettra de calculer leurs ages.".format(len(df_final)))

Après un nettoyage de donnée, nous obtenons une liste de 3124 présences d'athlètes toutes olympiades confondues, dont la date de naissance est connue; ce qui nous permettra de calculer leurs ages.


## Calcul de l'âge des athlètes lors de leurs particpations au jeux olympiques

In [23]:
#ajout d'une colonne donnant l'âge
df_final['age']= df_final.apply(lambda row : ( row['game_year'] - row['athlete_year_birth']), axis=1)
df_final = df_final.sort_values('age', ascending=True)

print(df_final.head(20)[['athlete_full_name_x','age']])
print("\n")
print(df_final.tail()[['athlete_full_name_x','age']])


              athlete_full_name_x   age
3134               William MORRIS -82.0
2564                   Alec SCOTT -61.0
2917                William SCOTT -40.0
3153               Jack A. WARNER -35.0
1987              Robert SULLIVAN -18.0
2474                William WHITE  -7.0
3268              Sidney ROBINSON  -2.0
3077                  William COX   4.0
120                     Sky BROWN  12.0
1496            James Guy PARRACK  12.0
1481            Zara Letitia LONG  14.0
121               Bombette MARTIN  14.0
14                   Thomas DALEY  14.0
2295        Charmain Isobel WELSH  15.0
1147             Sarah HARDCASTLE  15.0
1331           Susannah BROWNSDON  15.0
2002  Beverly Anna Maria WILLIAMS  15.0
1345                   Kathy READ  15.0
2594      Frederick George HODGES  15.0
2595        Elizabeth Joyce SLADE  15.0


         athlete_full_name_x   age
1806         Malcolm JENKINS  62.0
2239  Brenda Lilian WILLIAMS  65.0
1948   Hilda Lorna JOHNSTONE  66.0
2466           Cha

### Une fois le tri des ages par ordre croissant effectué, nous remarquons des valeurs incohérentes. Après vérification " Sky BROWN " est le plus jeune athlète britannique à avoir participer aux JO. Nous supprimons donc les données d'athlètes ayant un âge inférieur.
**Il ne semble pas y avoir d'anomalies concernant les athlètes très agés. "Hilda Lorna Johnstone" est, d'après la bio fournie avec les données, la compétitrice la plus âgée des JO.**

In [24]:
#Exploration et suppression de certains athlètes aux âges incohérents
df_final.drop(df_final.index[:8], inplace=True) 
print(df_final.head()[['athlete_full_name_x','age']])

     athlete_full_name_x   age
120            Sky BROWN  12.0
1496   James Guy PARRACK  12.0
1481   Zara Letitia LONG  14.0
121      Bombette MARTIN  14.0
14          Thomas DALEY  14.0


## Initialisation des graphiques

In [25]:
#graph1 : moyennes selon les différentes éditions des JO
df_mean_age = df_final.groupby(['game_year'])['age'].mean().reset_index(name="mean")
source1 = df_mean_age.from_records({
    'Année': df_mean_age['game_year'],
    'Moyenne': df_mean_age['mean']
})

graph1 = alt.Chart(source1).mark_bar().encode(
    x='Année', y='Moyenne').properties(
    title="Moyenne d'âge des participant par éditions",width=700,height=500)


#graph2 : tranches d'âge les plus représentés aux JO
df_group_age = df_final.groupby(['age']).size().reset_index(name="total")
source2 = df_group_age.from_records({
    'Age': df_group_age['age'],
    'Total': df_group_age['total']
})

graph2 = alt.Chart(source2).mark_bar().encode(
    x='Age', y='Total').properties(
    title="Répartition des participants par âges toutes olympiades confondues",width=700,height=500)

# Résultats :

In [26]:
graph1 & graph2

alt.VConcatChart(...)